## NLP Setup

In [1]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [2]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [3]:
from app.templates.meat_sale.deltas import meat_sale_deltas as deltas

s1 = deltas[0]
original_sentence = s1['original']
template_sentence = s1['general']
symboleo_template = s1['template'] 

print(f'- Original: {original_sentence}')
print(f'- General: {template_sentence}')
print(f'- Template: {symboleo_template}')

- Original: The Seller shall deliver the Order in one delivery within <delDueDateDays> days to the Buyer at its warehouse
- General: The Seller shall deliver the Order in one delivery [DELIVERY_TIMEFRAME] to the Buyer [DELIVERY_LOCATION]
- Template: delivery: O(seller, buyer, true, Happens(delivered))


In [4]:
from app.src.rules.meat_sale.delivery_timeframe.delivery_timeframe_config import DeliveryTimeframeConfig

config = DeliveryTimeframeConfig(nlp)

In [5]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer
from app.src.dynamic_constructor import DynamicObjectConstructor

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()

digraph = graph_builder.build(PAtom)

gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

dynamic_constructor = DynamicObjectConstructor(digraph)


In [14]:
#sentence = 'within 2 weeks'
#sentence = 'before April 25, 2022'
sentence = 'before the contract terminates'
doc = nlp(sentence)

summarizer.summarize(sentence)


Sentence: before the contract terminates

i     TEXT            POS             TAG             DEP             LEMMA           HEAD            ENT            
--------------------------------------------------------------------------------------------------------------
0     before          ADP             IN              mark            before          terminates                     
1     the             DET             DT              det             the             contract                       
2     contract        NOUN            NN              nsubj           contract        terminates      DOMAIN_EVENT   
3     terminates      VERB            VBZ             ROOT            terminate       terminates      DOMAIN_EVENT   



           SBAR                    
   _________|______                 
  |                S               
  |          ______|_________       
  |         NP               VP    
  |      ___|______          |      
  IN    DT         NN       VBZ   

c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [15]:
from app.src.rules.shared.main_processor import MainProcessor

dtf = DeliveryTimeframeProcessor(template, matcher, nlp, dynamic_constructor)

results = dtf.process(doc)

print(f'Original: {template.to_sym()}')
print(f'New: {results[0].to_sym()}')

case matches: {'before_event': before the contract terminates}
primitives:
- <class 'app.classes.spec.helpers.VariableDotExpression'> terminates
- <class 'app.classes.spec.sym_point.PointAtomContractEvent'> activated(self)
Original: Happens(delivered)
New: WhappensBeforeE(terminates, delivered)
